In [2]:
import xml.etree.ElementTree as ET
import html
from bs4 import BeautifulSoup
from dadata import DadataAsync
import requests
import pandas as pd
import numpy as np

In [83]:
# среднесписочная занятость
# https://www.nalog.ru/opendata/7707329152-sshr2019/
# after xmllint
root = ET.parse('emp.xml').getroot()
for type_tag in root.findall('Документ/СведССЧР'):
    value = type_tag.get('КолРаб')
    print(value)

1
1
4
5
4
1
2
12
4
1
3
1
1
3
4
1
0
1
17
6
1
1
3
91
1
1
2
3
9
1
1
3
3
3
0
1
5
1
2
1
3
1
1
1
1
11
1
1
0
1
0
0
1
17
1
1
1
0
1
5
26
2
17
1
1
1
50
2
2
2
1
0
1
1
1
1
2
1
8
4
1
1
1
1
1
15
1
32
3
2
2
4
1
0
14
15
1
1
1
2
4
9
1
2
1
1
30
5
1
10
1
6
1
1
1
1
0
1
1
2
1
0
2
1
1
1
1
1
3
1
5
0
1
4
0
2
0
3
1
1
24
5
1
3
1
10
2
6
1
1
1
0
0
1
1
3
0
1
5
1
1
1
1
7
1
7
1
1
2
0
1
1
5
1
2
1
5
3
2
0
8
2
16
1
1
0
0
2
1
1
0
0
8
1
1
1
1
1
1
2
2
3
1
1
1
1
3
2
1
1
1
1
1
0
4
0
4
0
6
0
2
1
1
1
1
5
1
2
5
3
1
1
7
1
2
0
0
4
1
10
73
2
9
3
1
1
1
5
0
2
1
1
0
1
1
5
2
1
10
1
1
7
46
1
3
4
2
1
2
2
1
1
7
12
4
2
1
1
0
4
2
2
1
1
0
5
3
4
1
2
3
0
1
1
1
10
1
1
1
1
3
1
1
1
1
1
6
1
1
0
1
0
1
14
2
1
1
6
1
5
4
1
4
28
3
1
1
2
1
1
1
2
4
1
0
1
1
2
1
0
6
2
2
1
1
1
0
6
12
1
1
1
0
3
1
1
2
1
1
0
3
1
3
2
3
8
1
1
4
1
90
1
0
1
4
4
2
6
1
2
4
4
1
1
3
2
2
0
1
3
6
9
2
1
2
2
0
2
5
3
2
1
1
2
0
1
3
0
1
1
1
2
2
3
0
1
2
1
1
10
51
1
2
1
1
2
1
1
4
0
0
2
2
1
14
1
2
1
4
1
4
22
1
0
1
2
1
1
1
0
1
3
11
1
2
1
1
1
0
1
7
1
1
4
2
1
1
4
3
2
1
23
1
0
0
1
1
16
1
1
0
1
0


In [36]:
token = "a324502f3a5cafa3217e9e43dd07b16d71b83050"
secret = "397d4dc269a188b3819401f5bff95f31f186d6f2"
dadata = DadataAsync(token, secret)


In [49]:
result = await dadata.suggest("party", "сбербанк")
[r for r in result]

In [80]:
df = pd.DataFrame()
url='https://soctrud.primorsky.ru/employer/?AreaFiasOktmo=9594&ShowWithoutVacancies=True&ShowProvidingQuota=True&Sort=13&Grid-pageSize=100&PageSize=10'
def parse_soup(soup):
    dl = []
    for i, tr in enumerate(soup.find_all("tr")):
        d = {}
        fields = [td.text for td in tr.children]
        d['name'], d['cat'], d['address'], d['emp'], d['vac'] = fields
        d['cat'] = d['cat'].replace('\t', '').replace('\n','').replace('\r','')
        dl += [d]
    return pd.DataFrame(dl[1:])
for i in range(1,20):
    r = requests.get(url + f'&Grid-page={i}')
    page = html.unescape(r.content.decode())
    soup = BeautifulSoup(page)
    df = df.append(parse_soup(soup), ignore_index=True)
    df.to_csv('emps.csv', sep='\t')
    
#     for j, td in enumerate(tr.children):
#         print(i, j, td.text)

ValueError: not enough values to unpack (expected 5, got 1)

In [56]:
df = pd.read_csv('emps.csv', sep='\t')
df

,Unnamed: 0,name,cat,address,emp,vac
0,0,"Филиал № 1 ООО ""МЕГАКОМ"" В Г.ВЛАДИВОСТОК",Торговля оптовая напитками…,"690039, г Владивосток, ул Русская, д. 46, корп. Б",57,3
1,1,"Филиал ООО ""Риквэст-Сервис""",Стирка и химическая чистка текстильных и мехов...,"690088, г Владивосток, ш Военное, д. 1, корп. А",93,2
2,2,"""Азиатско-Тихоокеанский Банк"" (ПАО)",Денежное посредничество прочее…,"690091, г Владивосток, ул Светланская, д. 56",209,16
3,3,"""ВЛАДИВОСТОКСКИЙ МОЛОЧНЫЙ КОМБИНАТ"" ФИЛИАЛ АО ...",Производство молока (кроме сырого) и молочной ...,"690087, г Владивосток, ул Стрелочная, д. 19",23,0
4,4,"""ВЛАДИВОСТОКСКИЙ"" ФИЛИАЛ ФГУП ""МОСКОВСКОЕ ПРОП...",Производство медицинских инструментов и оборуд...,"690068, г Владивосток, ул Магнитогорская, д. 8",62,3
...,...,...,...,...,...,...
1321,1321,ЧОУ ВМСОШ,Образование дошкольное…,"690041, г Владивосток, ул Горького, д. 10",42,2
1322,1322,"ЧОУ ДОД ""ЦЕНТР СПОРТИВНОЙ ПОДГОТОВКИ ""ОЛИМПИЕЦ""",Образование дополнительное детей и взрослых…,"690025, г Владивосток, ул Таежная, д. 8",37,0
1323,1323,"Чугуевский филиал КГКУ ""Приморское лесничество""",Лесоводство и прочая лесохозяйственная деятель...,"692623, р-н Чугуевский, с Чугуевка, ул Лесная,...",43,4
1324,1324,"ЧУЗ ""КБ ""РЖД-МЕДИЦИНА"" Г. ВЛАДИВОСТОК",Деятельность больничных организаций,"690003, г Владивосток, ул Верхнепортовая, д. 25",475,6


In [55]:
coords_ = coords[:]
coords_

[{'address': '690039, г Владивосток, ул Русская, д. 46, корп. Б',
  'lat': '43.166235',
  'lon': '131.913458',
  'district': 'Советский'},
 {'address': '690088, г Владивосток, ш Военное, д. 1, корп. А',
  'lat': '43.1329913',
  'lon': '131.9154604',
  'district': 'Первореченский'},
 {'address': '690091, г Владивосток, ул Светланская, д. 56',
  'lat': '43.113707',
  'lon': '131.8956449',
  'district': 'Ленинский'},
 {'address': '690087, г Владивосток, ул Стрелочная, д. 19',
  'lat': '43.133784',
  'lon': '131.9315649',
  'district': 'Первореченский'},
 {'address': '690068, г Владивосток, ул Магнитогорская, д. 8',
  'lat': '43.1769989',
  'lon': '131.9162445',
  'district': 'Советский'},
 {'address': '690068, г Владивосток, ул Кирова, д. 93',
  'lat': '43.182045',
  'lon': '131.9105',
  'district': 'Советский'},
 {'address': '692903, г Находка, пр-кт Находкинский, д. 86',
  'lat': '42.7877047',
  'lon': '132.8611598',
  'district': None},
 {'address': '690105, г Владивосток, ул Бородинск

In [37]:
coords = []
i = 0
for addr in df.address:
    # if i > 1 : break
    result = await dadata.clean("address", addr)
    # print(result)
    d = {'address':addr, 'lat':result['geo_lat'], 'lon':result['geo_lon'], 'district':result['city_district']}
    print(i, d)
    coords += [d]
    i+= 1
    

0 {'address': '690039, г Владивосток, ул Русская, д. 46, корп. Б', 'lat': '43.166235', 'lon': '131.913458', 'district': 'Советский'}
1 {'address': '690088, г Владивосток, ш Военное, д. 1, корп. А', 'lat': '43.1329913', 'lon': '131.9154604', 'district': 'Первореченский'}
2 {'address': '690091, г Владивосток, ул Светланская, д. 56', 'lat': '43.113707', 'lon': '131.8956449', 'district': 'Ленинский'}
3 {'address': '690087, г Владивосток, ул Стрелочная, д. 19', 'lat': '43.133784', 'lon': '131.9315649', 'district': 'Первореченский'}
4 {'address': '690068, г Владивосток, ул Магнитогорская, д. 8', 'lat': '43.1769989', 'lon': '131.9162445', 'district': 'Советский'}
5 {'address': '690068, г Владивосток, ул Кирова, д. 93', 'lat': '43.182045', 'lon': '131.9105', 'district': 'Советский'}
6 {'address': '692903, г Находка, пр-кт Находкинский, д. 86', 'lat': '42.7877047', 'lon': '132.8611598', 'district': None}
7 {'address': '690105, г Владивосток, ул Бородинская, д. 26', 'lat': '43.1649718', 'lon': '

59 {'address': '690105, г Владивосток, ул Русская, д. 65', 'lat': '43.170443', 'lon': '131.93509', 'district': 'Советский'}
60 {'address': '690001, г Владивосток, ул Пушкинская, д. 87', 'lat': '43.1152442', 'lon': '131.9126358', 'district': 'Ленинский'}
61 {'address': '690106, г Владивосток, пр-кт Красного Знамени, д. 10', 'lat': '43.1290083', 'lon': '131.8900767', 'district': None}
62 {'address': '690091, г Владивосток, пр-кт Океанский, д. 26, офис 503', 'lat': '43.1204491', 'lon': '131.8874196', 'district': 'Ленинский'}
63 {'address': '690091, г Владивосток, пр-кт Океанский, д. 17, офис 611', 'lat': '43.1194446', 'lon': '131.8864772', 'district': 'Фрунзенский'}
64 {'address': 'г Владивосток, ул Алеутская, д. 45, офис 503/1', 'lat': '43.1209118', 'lon': '131.8837968', 'district': 'Фрунзенский'}
65 {'address': '690062, г Владивосток, пр-кт Острякова, д. 44, корп. а', 'lat': '43.1350357', 'lon': '131.8989419', 'district': 'Фрунзенский'}
66 {'address': '690025, г Владивосток, ул Успенско

119 {'address': '690003, г Владивосток, ул Верхнепортовая, д. 5, корп. А', 'lat': '43.109055', 'lon': '131.88043', 'district': 'Фрунзенский'}
120 {'address': '690069, г Владивосток, ул Давыдова, д. 11', 'lat': '43.1767022', 'lon': '131.9212431', 'district': 'Советский'}
121 {'address': '690091, г Владивосток, ул Светланская, д. 67', 'lat': '43.1142243', 'lon': '131.8979889', 'district': 'Ленинский'}
122 {'address': '690912, г Владивосток, п Трудовое, ул Лермонтова, д. 84, Ориентир дом 84 (сразу за этим домом)', 'lat': '43.2948853', 'lon': '132.0599983', 'district': None}
123 {'address': '690105, г Владивосток, ул Русская, д. 73ж', 'lat': '43.167168', 'lon': '131.94122', 'district': 'Советский'}
124 {'address': '690017, г Владивосток, ул Патрокл, д. 27', 'lat': '43.0930646', 'lon': '131.9361871', 'district': 'Первомайский'}
125 {'address': '690001, г Владивосток, ул Дальзаводская, д. 27, корп. А', 'lat': '43.1120748', 'lon': '131.9151896', 'district': 'Ленинский'}
126 {'address': '69000

179 {'address': '690091, г Владивосток, ул Уборевича, д. 11, Собеседование по адерсу: г. Вл-ок ул. Тигровая, 7', 'lat': '43.1174145', 'lon': '131.8887675', 'district': 'Ленинский'}
180 {'address': '690003, г Владивосток, ул Станюковича, д. 29, корп. а', 'lat': '43.1067842', 'lon': '131.8745303', 'district': 'Фрунзенский'}
181 {'address': '692446, г Дальнегорск, пр-кт 50 лет Октября, д. 120', 'lat': '44.5582195', 'lon': '135.576117', 'district': None}
182 {'address': '692135, г Дальнереченск, ул 50 лет Октября, д. 81', 'lat': '45.9210918', 'lon': '133.7524392', 'district': None}
183 {'address': '690105, г Владивосток, ул Бородинская, д. 14', 'lat': '43.1637076', 'lon': '131.9187324', 'district': None}
184 {'address': '690091, г Владивосток, ул Светланская, д. 50', 'lat': '43.114083', 'lon': '131.8929391', 'district': 'Ленинский'}
185 {'address': '690013, г Владивосток, ул Флотская 1-я, д. 39, корп. А', 'lat': '43.1154747', 'lon': '131.9383605', 'district': 'Ленинский'}
186 {'address': '

239 {'address': '690011, г Владивосток, ул Новожилова, д. 3', 'lat': '43.0944379', 'lon': '131.9608167', 'district': 'Первомайский'}
240 {'address': '690080, г Владивосток, ул Борисенко, д. 104', 'lat': '43.08889', 'lon': '131.9632', 'district': 'Первомайский'}
241 {'address': '690091, г Владивосток, ул Светланская, д. 65, корп. а', 'lat': '43.1146491', 'lon': '131.8970913', 'district': 'Ленинский'}
242 {'address': '690091, г Владивосток, ул Батарейная, д. 2', 'lat': '43.1235489', 'lon': '131.8762423', 'district': 'Фрунзенский'}
243 {'address': '690080, г Владивосток, ул Борисенко, д. 102', 'lat': '43.090458', 'lon': '131.9657', 'district': 'Первомайский'}
244 {'address': '690002, г Владивосток, пр-кт Острякова, д. 49, офис 305', 'lat': '43.1375736', 'lon': '131.8995548', 'district': 'Фрунзенский'}
245 {'address': '690012, г Владивосток, ул Фастовская, д. 14, офис 419', 'lat': '43.1000578', 'lon': '131.8995418', 'district': 'Первомайский'}
246 {'address': '690091, г Владивосток, ул Бат

...

1058 {'address': '690091, г Владивосток, ул Луцкого, д. 12', 'lat': '43.1163371', 'lon': '131.8970243', 'district': 'Ленинский'}
1059 {'address': '690068, г Владивосток, пр-кт 100-летия Владивостока, д. 155, Цех №1 вход № 3 (справа от магазина "Винлаб")', 'lat': '43.1790701', 'lon': '131.9187624', 'district': 'Советский'}
1060 {'address': '690039, г Владивосток, ул Русская, д. 46', 'lat': '43.1666686', 'lon': '131.9128756', 'district': 'Советский'}
1061 {'address': '690063, г Владивосток, ул Приморская, д. 7, офис 1, 1 этаж;', 'lat': '43.0876675', 'lon': '131.898546', 'district': 'Первомайский'}
1062 {'address': '692243, г Спасск-Дальний, ул Складская, д. 2', 'lat': '44.6076323', 'lon': '132.8363841', 'district': None}
1063 {'address': '690105, г Владивосток, ул Русская, д. 94А', 'lat': '43.1728747', 'lon': '131.9389921', 'district': 'Советский'}
1064 {'address': '690091, г Владивосток, ул Пологая, д. 53, офис 411', 'lat': '43.121212', 'lon': '131.88693', 'district': None}
1065 {'addre

1119 {'address': '690091, г Владивосток, ул Фонтанная, д. 18, предварительно обращаться по телефону', 'lat': '43.1203079', 'lon': '131.8829865', 'district': 'Фрунзенский'}
1120 {'address': '690001, г Владивосток, ул Светланская, д. 89', 'lat': '43.1160402', 'lon': '131.9049799', 'district': 'Ленинский'}
1121 {'address': '690091, г Владивосток, ул Адмирала Фокина, д. 1', 'lat': '43.1194653', 'lon': '131.8797225', 'district': None}
1122 {'address': '690091, г Владивосток, ул Петра Великого, д. 2', 'lat': '43.113094', 'lon': '131.8927595', 'district': 'Ленинский'}
1123 {'address': '690091, г Владивосток, ул Фонтанная, д. 57', 'lat': '43.1197059', 'lon': '131.8899016', 'district': 'Ленинский'}
1124 {'address': '690106, г Владивосток, пр-кт Красного Знамени, д. 10, офис 617', 'lat': '43.1290083', 'lon': '131.8900767', 'district': None}
1125 {'address': '690074, г Владивосток, ул Снеговая, д. 39', 'lat': '43.1373497', 'lon': '131.9670799', 'district': 'Первореченский'}
1126 {'address': '6900

1180 {'address': '690911, г Владивосток, снт Лазурное снт, бухта Лазурная', 'lat': '43.1164904', 'lon': '131.8823937', 'district': 'Советский'}
1181 {'address': '690091, г Владивосток, ул Корабельная Набережная, д. 15, корп. а', 'lat': '43.112957', 'lon': '131.89357', 'district': 'Ленинский'}
1182 {'address': '690091, г Владивосток, ул Прапорщика Комарова, д. 54', 'lat': '43.1201642', 'lon': '131.8934501', 'district': 'Ленинский'}
1183 {'address': '692525, г Уссурийск, ул Пионерская, д. 19', 'lat': '43.8109561', 'lon': '131.9421323', 'district': None}
1184 {'address': '690033, г Владивосток, ул Иртышская, д. 2, корп. А', 'lat': '43.1600769', 'lon': '131.9135235', 'district': 'Первореченский'}
1185 {'address': '692446, г Дальнегорск, ул Инженерная, д. 8', 'lat': '44.5605986', 'lon': '135.5822449', 'district': None}
1186 {'address': '690091, г Владивосток, ул Уткинская, д. 36', 'lat': '43.1220393', 'lon': '131.891813', 'district': 'Ленинский'}
1187 {'address': '690922, г Владивосток, ост

1241 {'address': '692904, г Находка, ул Школьная, д. 1, корп. А', 'lat': '42.8268552', 'lon': '132.8927361', 'district': None}
1242 {'address': '690062, г Владивосток, ул Мыс Чумака, д. 1, корп. А', 'lat': '43.1385659', 'lon': '131.8924113', 'district': 'Фрунзенский'}
1243 {'address': '690033, г Владивосток, ул Гамарника, д. 3, корп. б', 'lat': '43.1628676', 'lon': '131.9171297', 'district': 'Первореченский'}
1244 {'address': '690034, г Владивосток, ул Фадеева, д. 47, корп. А', 'lat': '43.1092974', 'lon': '131.9784834', 'district': 'Ленинский'}
1245 {'address': '690080, г Владивосток, ул Командорская, д. 13, корп. А', 'lat': '43.0997924', 'lon': '131.9830543', 'district': 'Первомайский'}
1246 {'address': '690002, г Владивосток, ул Комсомольская, д. 9а', 'lat': '43.1307657', 'lon': '131.8910315', 'district': 'Фрунзенский'}
1247 {'address': '690002, г Владивосток, пр-кт Океанский, д. 88, корп. а, офис 304', 'lat': '43.1325974', 'lon': '131.899415', 'district': 'Фрунзенский'}
1248 {'addre

1301 {'address': '690090, г Владивосток, ул Уборевича, д. 20, корп. а, офис 214, 2 этаж; Наличие документа удостоверяющего личность;', 'lat': '43.1188124', 'lon': '131.8918393', 'district': 'Ленинский'}
1302 {'address': '690105, г Владивосток, ул Арсенальная, д. 15', 'lat': '43.1770708', 'lon': '131.9227375', 'district': 'Советский'}
1303 {'address': '690106, г Владивосток, пр-кт Партизанский, д. 28, корп. В', 'lat': '43.126289', 'lon': '131.8977603', 'district': 'Ленинский'}
1304 {'address': '690074, г Владивосток, ул Выселковая, д. 48', 'lat': '43.146285', 'lon': '131.9496867', 'district': 'Первореченский'}
1305 {'address': '690106, г Владивосток, пр-кт Партизанский, д. 28, корп. б', 'lat': '43.12618', 'lon': '131.89874', 'district': 'Ленинский'}
1306 {'address': '690105, г Владивосток, ул Арсенальная, д. 15', 'lat': '43.1770708', 'lon': '131.9227375', 'district': 'Советский'}
1307 {'address': '690105, г Владивосток, ул Арсенальная, д. 15', 'lat': '43.1770708', 'lon': '131.9227375', 

In [59]:
df.emp = df.emp.replace('н/д', 0).to_numpy(dtype=int)

In [60]:
df['lat'] = [d['lat'] for d in coords]
df['lon'] = [d['lon'] for d in coords]
df['district'] = [d['district'] for d in coords]
df.to_csv('emps.csv', sep='\t')
df

,Unnamed: 0,name,cat,address,emp,vac,lat,lon,district
0,0,"Филиал № 1 ООО ""МЕГАКОМ"" В Г.ВЛАДИВОСТОК",Торговля оптовая напитками…,"690039, г Владивосток, ул Русская, д. 46, корп. Б",57,3,43.166235,131.913458,Советский
1,1,"Филиал ООО ""Риквэст-Сервис""",Стирка и химическая чистка текстильных и мехов...,"690088, г Владивосток, ш Военное, д. 1, корп. А",93,2,43.1329913,131.9154604,Первореченский
2,2,"""Азиатско-Тихоокеанский Банк"" (ПАО)",Денежное посредничество прочее…,"690091, г Владивосток, ул Светланская, д. 56",209,16,43.113707,131.8956449,Ленинский
3,3,"""ВЛАДИВОСТОКСКИЙ МОЛОЧНЫЙ КОМБИНАТ"" ФИЛИАЛ АО ...",Производство молока (кроме сырого) и молочной ...,"690087, г Владивосток, ул Стрелочная, д. 19",23,0,43.133784,131.9315649,Первореченский
4,4,"""ВЛАДИВОСТОКСКИЙ"" ФИЛИАЛ ФГУП ""МОСКОВСКОЕ ПРОП...",Производство медицинских инструментов и оборуд...,"690068, г Владивосток, ул Магнитогорская, д. 8",62,3,43.1769989,131.9162445,Советский
...,...,...,...,...,...,...,...,...,...
1321,1321,ЧОУ ВМСОШ,Образование дошкольное…,"690041, г Владивосток, ул Горького, д. 10",42,2,43.2064851,131.9460228,Советский
1322,1322,"ЧОУ ДОД ""ЦЕНТР СПОРТИВНОЙ ПОДГОТОВКИ ""ОЛИМПИЕЦ""",Образование дополнительное детей и взрослых…,"690025, г Владивосток, ул Таежная, д. 8",37,0,43.2470288,132.0438035,Советский
1323,1323,"Чугуевский филиал КГКУ ""Приморское лесничество""",Лесоводство и прочая лесохозяйственная деятель...,"692623, р-н Чугуевский, с Чугуевка, ул Лесная,...",43,4,44.159503,133.881478,None
1324,1324,"ЧУЗ ""КБ ""РЖД-МЕДИЦИНА"" Г. ВЛАДИВОСТОК",Деятельность больничных организаций,"690003, г Владивосток, ул Верхнепортовая, д. 25",475,6,43.1040956,131.8713586,Фрунзенский


In [87]:
for district in set(df.district.values):
    print(f'{district}:\t {df[df.district.apply(lambda x: x == district)].emp.sum()}') 
    # df.district == None doesnt work

Первомайский:	 21446
Советский:	 85703
Фрунзенский:	 59048
Ленинский:	 58635
Первореченский:	 29968
None:	 47978


In [6]:
l =np.array([1,2,3,2,1,2,2,4,6]) 
l[np.where(l==2)[0]] = 15
l

array([ 1, 15,  3, 15,  1, 15, 15,  4,  6])